In [22]:

import importlib

# ENVIRONMENT
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import dotenv
import os

dotenv.load_dotenv('.env')
MT5_SERVER = os.environ["MT5_SERVER"]
MT5_LOGIN = os.environ["MT5_LOGIN"]
MT5_PASSWORD = os.environ["MT5_PASSWORD"]
DATA_PATH = os.environ["DATA_PATH"]
CATALOG_PATH = os.path.join(os.getcwd(), os.environ["CATALOG_PATH"])

# nautilus_trader imports

from nautilus_trader.model.identifiers import Venue, InstrumentId, Symbol
from nautilus_trader.model.data import Bar, BarType, QuoteTick
from nautilus_trader.config import BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.engine import BacktestResult
from nautilus_trader.trading.strategy import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos, maybe_unix_nanos_to_dt, unix_nanos_to_dt
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.cache.cache import Cache
from nautilus_trader.model.position import Position
from nautilus_trader.model.objects import Price
from decimal import Decimal

# other imports
from pandas import Timestamp
import importlib
import mplfinance as mpf
import matplotlib.pyplot as plt

# my packages
import put101.indicators as indicators
importlib.reload(indicators)

import strategies
importlib.reload(strategies)

import strategies.bollinger_cluster
importlib.reload(strategies.bollinger_cluster)
from strategies.bollinger_cluster import BollingerCluster


import put101.utils as utils
importlib.reload(utils)


# ---------------- CONFIGURATION ----------------
catalog = ParquetDataCatalog(CATALOG_PATH)
start = dt_to_unix_nanos(pd.Timestamp("2023-11-01 00:00:00"))
end = start + pd.Timedelta(days=30).value 

venue_str = "SIM_EIGHTCAP"
venue = Venue(venue_str)
symbol_str = "EURUSD"
symbol = Symbol(symbol_str)
instrument_id_str = f"EURUSD.{venue}"

instrument_id = InstrumentId(symbol, venue)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
venue_configs = [
    BacktestVenueConfig(
        name=venue_str,
        oms_type="HEDGING",
        account_type="MARGIN",
        base_currency="USD",
        starting_balances=["10_000 USD"],
    ),
]

data_configs = [
    BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=QuoteTick,
        instrument_id=instrument_id,
        start_time=start,
        end_time=end,
    ),
]

strategies = [
    ImportableStrategyConfig(
        strategy_path="strategies.bollinger_cluster:BollingerCluster",
        config_path="strategies.bollinger_cluster:BollingerClusterConfig",
        config=dict(
            instrument_id=instrument_id.value,
            bar_type=f"{instrument_id}-15-MINUTE-BID-INTERNAL",
            bb_params=[
                (5, 2),
                (60, 2),
            ],
        ),
    ),
]

configs = [BacktestRunConfig(
    engine=BacktestEngineConfig(
        strategies=strategies,
    ),
    data=data_configs,
    venues=venue_configs,
)]

node = BacktestNode(configs)
print(strategies)

[autoreload of strategies.bollinger_cluster failed: Traceback (most recent call last):
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "/Users/tobiaspucher/GitHub/nautilus/.venv/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: can't apply this __setattr__

[ImportableStrategyConfig(strategy_path='strategies.bollinger_cluster:BollingerCluster', config_path='strategies.bollinger_cluster:BollingerClusterConfig', config={'instrument_id': 'EURUSD.SIM_EIGHTCAP', 'bar_type': 'EURUSD.SIM_EIGHTCAP-15-MINUTE-BID-INTERNAL', 'bb_params': [(5, 2), (60, 2)]})]


In [24]:
results = node.run()

2023-11-01T17:15:00.286000000Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2023-11-01T17:15:00.286000000Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2023-11-01T17:15:00.286000000Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2023-11-01T17:15:00.286000000Z [INFO] BACKTESTER-001.BacktestEngine: Copyright (C) 2015-2024. All rights reserved.
2023-11-01T17:15:00.286000000Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2023-11-01T17:15:00.286000000Z [INFO] BACKTESTER-001.BacktestEngine: 
2023-11-01T17:15:00.286000000Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2023-11-01T17:15:00.286000000Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2023-11-01T17:15:00.286000000Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2023-11-01T17:15:

In [ ]:
res = results[0]
backtest_start = maybe_unix_nanos_to_dt(res.backtest_start)
backtest_end = maybe_unix_nanos_to_dt(res.backtest_end)
res

In [ ]:
import put101.vizz as vizz

# This allows multiple outputs from a single jupyter notebook cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib qt


engine = node.get_engine(res.run_config_id)
strategy: BollingerCluster = engine.trader.strategies()[0]
cache: Cache = strategy.cache

main_t, main_s = strategy.get_main_plottable_indicators()
extra_plots = strategy.get_extra_plots()

layout = utils.get_layout(
                    res=res,
                    bars=strategy.bars,
                    overlay_indicators=main_t,
                    overlay_indicator_styles=main_s,
                    extra_plots=extra_plots,
                    positions=strategy.cache.positions(),
)

vizz.reset_output()
vizz.show(layout)

In [ ]:
res

In [ ]:
pd.set_option("display.max_colwidth", None)
positions = strategy.cache.positions()
df = pd.DataFrame([p.to_dict() for p in positions])

maybe_unix_nanos_to_dt(df["ts_opened"][1])

df.iloc[1]

In [ ]:
strategy.portfolio_equity

In [ ]:
from nautilus_trader.model.instruments import Instrument
Instrument.base_to_dict(strategy.instrument)


In [ ]:
positions = strategy.cache.positions()
df = pd.DataFrame([p.to_dict() for p in positions])
print(df)

In [ ]:
ins = strategy.instrument
p = ins.make_price(Decimal(1.12000))
r = ins.make_price(Decimal(1.12055))
q = ins.make_qty(Decimal(1123.123))

In [ ]:
p = ins.make_price(Decimal(1.12000))
r = ins.make_price(Decimal(1.12055))
p.raw
r.raw

pip_risk = 55 #(r-p)
point_value_per_unit = ins.price_increment * ins.lot_size

acc_risk =  Decimal(100)
lots = (acc_risk / pip_risk) * (1 / point_value_per_unit)
qty = lots * ins.lot_size
qty = ins.make_qty(qty)

class RiskCalculator:

    @staticmethod
    def qty_from_risk(risk: Decimal, entry: Decimal, exit: Decimal, ins: Instrument) :
        risk_points = Decimal( abs(entry - exit))
        print(f"risk_points: {risk_points}")
        point_value_per_unit = ins.price_increment * ins.lot_size
        lots = (risk / risk_points) * (1 / point_value_per_unit)
        qty = lots * ins.lot_size
        return ins.make_qty(qty)
    

RiskCalculator.qty_from_risk(Decimal(100), Decimal(1.12000), Decimal(1.12055), ins)